In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

ROOT_DIR = os.path.abspath('../..')
sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)


# Suppress all future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from transformers import logging

from tqdm import tqdm
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\FS-Ma\miniconda3\envs\neuron\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from neuron.runtime_logging import start
from neuron.agents import UserAgent
from neuron.agents.agents import LearnerAgent, KnowledgeRepresenterAgent
from neuron.agents.llm_agents import PerceiverAgent
from neuron.scenarios import RoundRobin, RoundRobinManager

from neuron.capabilities.perceiver import SemanticTemplateFillerCapability

# Start logging
logging_session_id = start(config={"dbname": "logs.db"})
print("Logging session ID: " + str(logging_session_id))

Logging session ID: 3d3ae119-459e-4ce1-8d3a-40e249814f92


In [3]:
import pandas as pd
df = pd.read_json("C:/Users/FS-Ma/OneDrive/Documents/projects/neuron/data/yelp_academic_dataset_business.json", lines=True, orient='columns', chunksize=1000000)
# read the data 
for business in df:
    business = business
    break
business.drop(['business_id'], axis=1, inplace=True)

In [4]:
llm_config={
    "config_list": [{
        "client": "groq",
        "model": "llama3-groq-70b-8192-tool-use-preview",
        "api_key": "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"
        }
]}

user_agent = UserAgent(
    name="User Agent",
    llm_config=llm_config,
)

perceiver_agent = PerceiverAgent(
    llm_config=llm_config,
)

leaner_agent = LearnerAgent(
    llm_config = {
    "config_list": [{
        "client": "Begreat",
        "model": "distilgpt2",
        "model_dir": "C:/Users/FS-Ma/OneDrive/Documents/projects/neuron/models/be_great_yelp",
    }]},
)

knowledge_representer_agent = KnowledgeRepresenterAgent(
    description="Responsible to generate representations of the information.",
    llm_config= {
        "config_list": [{
            "client": "LLMEmbedding",
            "model_dir": "C:/Users/FS-Ma/OneDrive/Documents/projects/neuron/models/erasmo_yelp_gpt2-medium_60_True",
            "model": "gpt2-medium"
    }]},
)

semantic_template_filler = SemanticTemplateFillerCapability(row=business.iloc[0], llm_config=llm_config)
semantic_template_filler.add_to_agent(perceiver_agent)

groupchat = RoundRobin(agents=[user_agent, perceiver_agent, leaner_agent, knowledge_representer_agent], messages=[])
manager = RoundRobinManager(groupchat=groupchat, llm_config=None)

user_agent.initiate_chat(
    manager, message="I like sushi e meu nome é sildofo e meus reviews_count são 10 eu sou gosto de restaurantes com 10 estrelas."
)

User Agent (to chat_manager):

I like sushi e meu nome é sildofo e meus reviews_count são 10 eu sou gosto de restaurantes com 10 estrelas.

--------------------------------------------------------------------------------
perceiver_agent (to chat_manager):

{'name': ['sildofo'], 'address': [np.nan], 'city': [np.nan], 'state': [np.nan], 'postal_code': [np.nan], 'latitude': [np.nan], 'longitude': [np.nan], 'stars': ['10'], 'review_count': ['10'], 'is_open': [np.nan], 'attributes': [np.nan], 'categories': [np.nan], 'hours': [np.nan]}

--------------------------------------------------------------------------------


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


learner_agent (to chat_manager):

name is sildofo, address is 5130 W University Rd, city is Madison, state is placeholder, postal_code is placeholder, latitude is 33.824804, longitude is -112.725514, stars is 10.0, review_count is 10.0, is_open is 1.0, attributes is {'OutdoorSeating': 'False', categories is Restaurants, hours is placeholder

--------------------------------------------------------------------------------
